(section-scf)=
# Wave function optimization

## Import VeloxChem

VeloxChem is invoked with a Python module import.

In [1]:
import veloxchem as vlx
# On Windows computers activate the lines below
# import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.


## Define the molecule

The molecular structure is commonly expressed as xyz-coordinates, and need to be obtained from experimental measurements, calculations, or built manually. One approach is to use the build the molecule by construcing a [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) string defining the connectivities, and pass this to a package such as [rdkit](https://www.rdkit.org/) which can then construct the molecule.The is discussed more on [eChem](https://kthpanor.github.io/echem/docs/tutorials/wf_build.html), and for now we note that the SMILES strings are often available on, *e.g.* wikipedia, with that of [caffeine](https://en.wikipedia.org/wiki/Caffeine) being:

```
CN1C=NC2=C1C(=O)N(C(=O)N2C)C
```

Using this to build a conformer, visualize the structure, and print the xyz-coordinates:

In [2]:
import py3Dmol as p3d
from rdkit import Chem
from rdkit.Chem import AllChem

def smilestoxyz(smiles):
    mol_bare = Chem.MolFromSmiles(smiles)
    mol_full = Chem.AddHs(mol_bare)
    AllChem.EmbedMolecule(mol_full)
    AllChem.UFFOptimizeMolecule(mol_full)
    return Chem.MolToXYZBlock(mol_full)

caffeine_xyz = smilestoxyz('CN1C=NC2=C1C(=O)N(C(=O)N2C)C')

viewer = p3d.view(width=300, height=200)
viewer.addModel(caffeine_xyz, 'xyz')
viewer.setViewStyle({"style": "outline", "color": "black", "width": 0.1})
viewer.setStyle({"stick": {}})
viewer.show()

print(caffeine_xyz)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

24

C      3.263017    0.665602   -0.212720
N      2.130944   -0.255164   -0.199731
C      2.169427   -1.598733   -0.377053
N      0.942209   -2.179661   -0.328611
C      0.122902   -1.131387   -0.108457
C      0.826777    0.020965   -0.033402
C      0.176416    1.232141    0.162798
O      0.830662    2.309769    0.212003
N     -1.194119    1.221596    0.291480
C     -1.895573    0.031642    0.181110
O     -3.158218    0.040671    0.233363
N     -1.232449   -1.164517    0.007067
C     -1.967842   -2.428192   -0.134534
C     -1.923424    2.489165    0.463343
H      4.214642    0.108953   -0.348684
H      3.149265    1.386854   -1.048964
H      3.309011    1.216933    0.749769
H      3.081659   -2.156591   -0.545352
H     -1.328558   -3.298924    0.124931
H     -2.841765   -2.451853    0.551042
H     -2.320805   -2.537641   -1.181510
H     -2.839017    2.343970    1.075630
H     -2.209018    2.891819   -0.531182
H     -1.306144    3.242585    0.997663



Moving to a smaller system, we define the structure of a water molecule and choose a basis set.

In [3]:
# Atomic coordinates in units of Angstrom
water_xyz = """3 

O       0.0000000000     0.1178336003     0.0000000000
H      -0.7595754146    -0.4713344012    -0.0000000000
H       0.7595754146    -0.4713344012     0.0000000000
"""

molecule = vlx.Molecule.from_xyz_string(water_xyz)
basis = vlx.MolecularBasis.read(molecule, "cc-pVDZ")

* Info * Reading basis set from file: /home/thomas/Notebook/anaconda/envs/echem_test/lib/python3.9/site-packages/veloxchem/basis/CC-PVDZ
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: CC-PVDZ                                                                             
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                                                                                                          
  

In [4]:
print('Number of atoms:', molecule.number_of_atoms())
print('Number of electrons:', molecule.number_of_electrons())
print('Number of contracted basis functions:', basis.get_dimensions_of_basis(molecule))

Number of atoms: 3
Number of electrons: 10
Number of contracted basis functions: 24


## SCF optimization

Perform a self-consistent field (SCF) optimization to obtain the Hartree–Fock wave function and the associated ground-state energy.

In [5]:
scf_drv = vlx.ScfRestrictedDriver()
scf_results = scf_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

(section-scf-info)=
## SCF information

The SCF driver object has a method named `get_scf_energy()` for retrieving the final energy.

In [6]:
print(f'Hartree–Fock energy: {scf_drv.get_scf_energy():14.10f} a.u.')

Hartree–Fock energy: -76.0265782198 a.u.


The return object from the `compute()` method is a Python dictionary containing several tensors:

- `C`: molecular orbital coefficients as a NumPy array
- `E`: orbital energies as a NumPy array
- `D`: $\alpha$- and $\beta$-spin density matrices as a tuple of NumPy arrays
- `F`: $\alpha$- and $\beta$-spin Fock matrices as a tuple of NumPy arrays
- `S`: overlap integrals as a NumPy array

In [7]:
print('Dictionary keys:\n', scf_drv.scf_tensors.keys())
print()
print('Orbital energies:\n', scf_drv.scf_tensors['E'])

Dictionary keys:
 dict_keys(['S', 'C_alpha', 'C_beta', 'E_alpha', 'E_beta', 'D_alpha', 'D_beta', 'F_alpha', 'F_beta', 'C', 'E', 'D', 'F'])

Orbital energies:
 [-20.55119961  -1.33473347  -0.69692651  -0.56605275  -0.49289827
   0.18486487   0.25566978   0.78604709   0.85103488   1.16387193
   1.20030818   1.25366153   1.44422793   1.4755769    1.67329727
   1.8679223    1.93111284   2.4430781    2.48048696   3.28268216
   3.334531     3.50569214   3.86065795   4.14355137]
